In [13]:
import os

while os.getcwd()[-len('mcrc-cetuximab-analysis'):] != 'mcrc-cetuximab-analysis':
    os.chdir('..')
os.getcwd() 

'/home/max/mcrc-cetuximab-analysis'

In [14]:
import os
import GEOparse as Geo

In [15]:
import pandas as pd

def decouple(s):
    ls = list(s.split("', '"))
    if len(ls) == 3:
        ls[0] = ls[0][2:]
        ls[2] = ls[2][:-2]
    elif len(ls) == 2:
        ls[0] = ls[0][2:]
        ls[1] = ls[1][:-2]
    dc = {s.split(': ')[0]: s.split(': ')[1] for s in ls}
    return dc

def extend_id(id):
    fp, sp = id.split('_')
    while len(sp) < 4:
        sp = '0' + sp
    return fp + '_' + sp

def pad_zeros(s, n):
    while len(s) < n:
        s = '0' + s
    return s

gse = Geo.get_GEO(filepath="raw/GSE183984_family.soft", silent=True)
print('Platform: ', gse.gpls)
for key in gse.gsms:
    ann = pd.DataFrame(columns=gse.gsms[key].metadata.keys())
    break

for key in gse.gsms:
    ann = pd.concat([ann, pd.DataFrame([gse.gsms[key].metadata], index=[key], dtype=str)])

print('Annotation size after collecting it from .SOFT: ', len(ann))

new_ann = pd.DataFrame(columns=['sample_id', 'tissue', 'time point', 'treatment response', 'patient_id', 'gsm_id', 'sample_title']).set_index('gsm_id')
for i in range(len(ann)):
    new_row = decouple(str(ann.iloc[i]['characteristics_ch1']))
    # print(new_row)
    title = str(ann.iloc[i]['title'])
    new_row['sample_id'] = extend_id(title[title.find('[', 1) + 1 : title.find(']')])
    new_row['sample_title'] = title[2:title.find(' ')]
    new_row['patient_id'] = (lambda s: pad_zeros(''.join(filter(lambda x: x.isdigit(), s)), 4))(new_row['sample_title'])
    new_ann = pd.concat([new_ann, pd.DataFrame([new_row], index=[ann.index[i]])])

ann = new_ann
ann['gsm_id'] = ann.index
ann = ann.set_index('sample_id')

print('Annotation size after creating new columns: ', len(ann))

ann = ann.loc[~ann["tissue"].str.contains("metastases", case=False, na=False)]
print('Annotation size after deleting metastatic samples: ', len(ann))


print('Patient 5 indexes: ', ann.loc[ann['patient_id'] == '0005'].index)
ann = ann.loc[ann['patient_id'] != '0005']
print('Annotation size after deleting patient 0005: ', len(ann))


print('Nans: ', ann.isna().sum())

# setting NaN to pre because it is known from another column (treatment response)
ann.loc[ann['time point'].isna(), 'time point'] = 'pre-treatment'

def convert_treatment_response(s):
    if 'non-PD' in s:
        return 'non-PD'
    elif 'PD' in s:
        return 'PD'
    elif s == 'pre-Tx':
        return 'unknown'
    else:
        raise Exception('unexpected treatment response in annotation row')

ann['treatment response'] = ann['treatment response'].apply(convert_treatment_response) 

treatment_response_for_patient = dict()

for i in range(len(ann)):
    row = ann.iloc[i]
    patient_id = row['patient_id']
    response = row['treatment response']
    time_point = row['time point']
    
    if time_point == 'pre-treatment' and response != 'unknown':
        raise Exception('known response in pre-treatment')
    
    if time_point == 'post-treatment' and response == 'unknown':
        raise Exception('unknown response in post-treatment')
    
    if response != 'unknown':
        if patient_id in treatment_response_for_patient and response != treatment_response_for_patient[patient_id]:
            raise Exception('contradictive post-treatment response for patient')
        
        treatment_response_for_patient[patient_id] = response

for i in range(len(ann)):
    row = ann.iloc[i]
    patient_id = row['patient_id']
    response = row['treatment response']
    
    if response == 'unknown' and patient_id in treatment_response_for_patient:
        ann.loc[ann.index[i],'treatment response'] = treatment_response_for_patient[patient_id]



Platform:  {'GPL16791': <d: GPL16791>}
Annotation size after collecting it from .SOFT:  113
Annotation size after creating new columns:  113
Annotation size after deleting metastatic samples:  85
Patient 5 indexes:  Index(['18R689_0022', '18R690_0023', '18R697_0005', '18R176_0020'], dtype='object', name='sample_id')
Annotation size after deleting patient 0005:  81
Nans:  tissue                0
time point            1
treatment response    0
patient_id            0
sample_title          0
gsm_id                0
dtype: int64


In [16]:
ann.to_csv('data/ann_unfiltered.csv', index=True)

In [17]:

ann = ann.drop(['18R294_0001'])
# this is from patient 0005
# ann = ann.drop(['18R697_0005',])
print('Annotation size after deleting 18R697_0005, 18R294_0001: ', len(ann))


Annotation size after deleting 18R697_0005, 18R294_0001:  80


In [18]:
ann.to_csv('data/ann.csv', index=True)

In [19]:
ann.index

Index(['18R649_0010', '18R654_0015', '18R669_0001', '18R670_0002',
       '18R674_0006', '18R678_0010', '18R683_0015', '18R694_0002',
       '18R699_0007', '18R700_0008', '18R429_0009', '18R298_0010',
       '18R647_0008', '18R648_0009', '18R295_0003', '18R296_0008',
       '18R178_0022', '18R177_0021', '18R299_0025', '18R300_0027',
       '18R659_0020', '18R661_0021', '18R301_0002', '18R302_0004',
       '18R663_0027', '18R664_0001', '18R307_0013', '18R319_0014',
       '18R685_0018', '18R642_0003', '18R645_0006', '18R653_0014',
       '18R432_0012', '18R434_0014', '18R460_0001', '18R695_0003',
       '18R473_0014', '18R443_0022', '18R436_0016', '18R421_0001',
       '18R463_0004', '18R472_0013', '18R426_0006', '18R427_0007',
       '18R508_0011', '18R433_0013', '18R435_0015', '18R437_0018',
       '18R449_0023', '18R438_0019', '18R440_0020', '18R456_0002',
       '18R457_0023', '18R458_0025', '18R459_0027', '18R461_0002',
       '18R462_0003', '18R464_0005', '18R468_0009', '18R469_00

In [20]:
ann['tissue'].value_counts()

tissue
primary tumor (colorectum)    80
Name: count, dtype: int64

In [21]:
ann['time point'].value_counts()

time point
pre-treatment     67
post-treatment    13
Name: count, dtype: int64

In [22]:
ann['treatment response'].value_counts()

treatment response
unknown    42
non-PD     27
PD         11
Name: count, dtype: int64

In [23]:
ann['patient_id'].value_counts().sort_index()

patient_id
0001    3
0002    3
0003    1
0004    3
0006    3
0007    1
0008    3
0009    1
0010    3
0011    2
0012    3
0013    3
0014    3
0015    4
0016    2
0017    1
0018    1
0019    1
0020    2
0021    3
0022    1
0023    1
0024    1
0026    1
0027    1
0028    1
0029    2
0030    1
0031    1
0032    2
0033    1
0034    1
0035    1
0036    1
0037    1
0038    1
0040    1
0041    1
0042    1
0043    1
0044    1
0045    1
0046    1
0047    1
0048    1
0049    1
0050    1
0051    1
0052    1
0054    1
0055    1
0061    1
Name: count, dtype: int64

In [24]:
ann[ann['time point'] == 'post-treatment']['treatment response'].value_counts()

treatment response
non-PD    9
PD        4
Name: count, dtype: int64

In [25]:
ann[ann['time point'] == 'pre-treatment']['treatment response'].value_counts()

treatment response
unknown    42
non-PD     18
PD          7
Name: count, dtype: int64